In [1]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import numpy as np
from argparse import ArgumentParser
from double_net.double_net import DoubleNet, train_loop, train_loop_no_lagrange, test_loop
from double_net import datasets as ds
import time
import double_net.plot_utils as pu
import importlib
import matplotlib.pyplot as plt

In [2]:
if torch.cuda.is_available():
    device = 'cuda'
else:
    device = 'cpu'

In [3]:
parser = ArgumentParser()
parser.add_argument('--random-seed', type=int, default=0)
parser.add_argument('--num-examples', type=int, default=131072 * 4)
parser.add_argument('--test-num-examples', type=int, default=1000)
parser.add_argument('--test-iter', type=int, default=5)
parser.add_argument('--n-agents', type=int, default=2)
parser.add_argument('--n-items', type=int, default=3)
parser.add_argument('--num-epochs', type=int, default=100)
parser.add_argument('--batch-size', type=int, default=128 * 32)
parser.add_argument('--test-batch-size', type=int, default=1000)
parser.add_argument('--model-lr', type=float, default=1e-3)
parser.add_argument('--misreport-lr', type=float, default=1e-1)
parser.add_argument('--misreport-iter', type=int, default=25)
parser.add_argument('--test-misreport-iter', type=int, default=1000)
parser.add_argument('--rho', type=float, default=1)
parser.add_argument('--rho-incr-iter', type=int, default=2)
parser.add_argument('--rho-incr-amount', type=float, default=1.0)
parser.add_argument('--lagr-update-iter', type=int, default=100)
parser.add_argument('--rgt-start', type=int, default=0)
parser.add_argument('--sinkhorn-epsilon', type=float, default=5e-2)
parser.add_argument('--sinkhorn-rounds', type=int, default=20)


args = parser.parse_args(args=[])
args

Namespace(batch_size=4096, lagr_update_iter=100, misreport_iter=25, misreport_lr=0.1, model_lr=0.001, n_agents=2, n_items=3, num_epochs=100, num_examples=524288, random_seed=0, rgt_start=0, rho=1, rho_incr_amount=1.0, rho_incr_iter=2, sinkhorn_epsilon=0.05, sinkhorn_rounds=20, test_batch_size=1000, test_iter=5, test_misreport_iter=1000, test_num_examples=1000)

In [4]:
torch.manual_seed(args.random_seed)
np.random.seed(args.random_seed)

item_ranges = ds.preset_valuation_range(args.n_agents, args.n_items, "")

model = DoubleNet(
    args.n_agents, args.n_items, item_ranges, args.sinkhorn_epsilon, args.sinkhorn_rounds, marginal_choice='additive'
).to(device)

train_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.num_examples, item_ranges).to(device)
train_loader = ds.Dataloader(train_data, batch_size=args.batch_size, shuffle=True)

In [5]:
%%time
train_loop(model, train_loader, args, device=device)

  1%|          | 1/100 [01:09<1:55:08, 69.78s/it]

{'regret_max': 0.7837794423103333, 'regret_mean': 0.030194148421287537, 'regret_mults': tensor([[6.0785, 5.9748]], device='cuda:0'), 'payment': 1.2467942237854004}


  2%|▏         | 2/100 [02:18<1:53:13, 69.32s/it]

{'regret_max': 0.2061464786529541, 'regret_mean': 0.011071741580963135, 'regret_mults': tensor([[7.2694, 7.0683]], device='cuda:0'), 'payment': 1.3274056911468506}


  3%|▎         | 3/100 [03:20<1:48:53, 67.35s/it]

...

{'regret_max': 0.0260465145111084, 'regret_mean': 0.0002087180910166353, 'regret_mults': tensor([[192.7953, 198.3011]], device='cuda:0'), 'payment': 1.2219252586364746}


 99%|█████████▉| 99/100 [1:48:46<01:08, 68.34s/it]

{'regret_max': 0.031783461570739746, 'regret_mean': 0.0002479145477991551, 'regret_mults': tensor([[193.3525, 199.5122]], device='cuda:0'), 'payment': 1.223276138305664}


100%|██████████| 100/100 [1:49:49<00:00, 65.89s/it]

{'regret_max': 0.03066706657409668, 'regret_mean': 0.0003167768591083586, 'regret_mults': tensor([[195.1678, 202.0174]], device='cuda:0'), 'payment': 1.2201862335205078}
CPU times: user 1h 49min 26s, sys: 16 s, total: 1h 49min 42s
Wall time: 1h 49min 49s


In [6]:
test_data = torch.load('./additive_2x3_test_data')

In [7]:
%%time
# model.sinkhorn_rounds = 100
# model.sinkhorn_epsilon = 1e-2
# test_data = ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device)
# cpu_test_data = test_data.clone()
test_data = torch.load('./additive_2x3_test_data')
test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop(model, test_loader, args, device=device)
result

1it [00:18, 18.19s/it]

CPU times: user 18.1 s, sys: 49 ms, total: 18.2 s
Wall time: 18.2 s


{'payment_mean': 1.1748871803283691,
 'regret_mean': 0.001791558344848454,
 'regret_max': 0.014467060565948486}

In [8]:
from tqdm import tqdm
from double_net import utils_misreport as utils
def test_loop_random_start(model, loader, args, random_starts, device='cpu'):
    # regrets and payments are 2d: n_samples x n_agents; unfairs is 1d: n_samples.
    test_regrets = torch.Tensor().to(device)
    test_payments = torch.Tensor().to(device)

    for i, batch in tqdm(enumerate(loader)):
        batch = batch.to(device)
        allocs, payments = model(batch)
        truthful_util = utils.calc_agent_util(batch, allocs, payments)
        
        max_regrets = torch.zeros(truthful_util.shape).to(device=device)
        for c in range(len(random_starts)):
            misreport_batch = random_starts[c].clone().detach()
            utils.optimize_misreports(model, batch, misreport_batch,
                                  misreport_iter=args.test_misreport_iter, lr=args.misreport_lr)
            misreport_util = utils.tiled_misreport_util(misreport_batch, batch, model)

            regrets = misreport_util - truthful_util
            positive_regrets = torch.clamp_min(regrets, 0)
            max_regrets = torch.max(max_regrets, positive_regrets)

        # Record entire test data
        test_regrets = torch.cat((test_regrets, max_regrets), dim=0)
        test_payments = torch.cat((test_payments, payments), dim=0)

    mean_regret = test_regrets.sum(dim=1).mean(dim=0).item()
    result = {
        "payment_mean": test_payments.sum(dim=1).mean(dim=0).item(),
        "regret_mean": mean_regret,
        "regret_max": test_regrets.sum(dim=1).max().item(),
    }
    return result

In [9]:
%%time
args.test_num_examples = 1000
args.test_batch_size = 1000
args.test_misreport_iter = 1000
args.misreport_lr = 1e-1
model.sinkhorn_rounds = 20
model.sinkhorn_epsilon = 3e-2

random_starts = [test_data]
for i in range(10):
    random_starts.append(ds.generate_dataset_nxk(args.n_agents, args.n_items, args.test_num_examples, item_ranges).to(device))
test_loader = ds.Dataloader(test_data, batch_size=args.test_batch_size, shuffle=True)

result = test_loop_random_start(model, test_loader, args, random_starts, device=device)
result

1it [03:25, 205.16s/it]

CPU times: user 3min 24s, sys: 419 ms, total: 3min 24s
Wall time: 3min 25s


{'payment_mean': 1.2150055170059204,
 'regret_mean': 0.019259242340922356,
 'regret_max': 0.06496179848909378}

In [10]:
torch.save(test_data, './additive_2x3_test_data')

In [11]:
model.save('./additive_one_2x3_0_1_')

In [12]:
import cvxpy as cp
import numpy as np
from typing import NamedTuple
from tqdm.notebook import tqdm

In [13]:
class AuctionResult(NamedTuple):
    alloc: np.ndarray
    welfare: np.ndarray
    payment: np.ndarray

In [14]:
def max_welfare_alloc(bids_mat, k, exact_demand=True):
    # returns allocation, util of allocation

    num_agents = bids_mat.shape[0]
    num_items = bids_mat.shape[1]
    alloc_matrix = cp.Variable((num_agents, num_items), nonneg=True)
    item_supply_constraints = cp.sum(alloc_matrix, axis=0) <= 1
    if exact_demand:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) == k
    else:
        agent_demand_constraints = cp.sum(alloc_matrix, axis=1) <= k
    welfare = cp.sum((cp.multiply(bids_mat, alloc_matrix)))
    problem = cp.Problem(cp.Maximize(welfare), [item_supply_constraints, agent_demand_constraints])
    problem.solve()
    return (alloc_matrix.value, problem.value)

In [15]:
def vcg_auction(bids_mat, k, exact_demand=False):
    main_alloc, max_welfare_total_util = max_welfare_alloc(bids_mat, k, exact_demand=exact_demand)
    payments = np.zeros(bids_mat.shape[0])
    player_utils = (bids_mat * main_alloc).sum(axis=1)
    num_agents = bids_mat.shape[0]
    for i in range(num_agents):
        dropped_bid_mat = np.delete(bids_mat, (i), axis=0)
        dropped_player_utils = np.delete(player_utils, (i), axis=0) # player utils under full auction
        new_alloc, new_total_util = max_welfare_alloc(dropped_bid_mat, k, exact_demand=exact_demand)
        new_agent_utils = (new_alloc*dropped_bid_mat).sum(axis=1) # player utils without agent i's bid
        payments[i] = (new_agent_utils - dropped_player_utils).sum() 
    return AuctionResult(main_alloc, player_utils, payments)

In [17]:
payments = []
for i in range(test_data.shape[0]):
    payments.append(vcg_auction(test_data[i].cpu().numpy(), 3, exact_demand=False).payment.sum())

In [18]:
np.mean(payments)

0.9816638948070513

In [ ]:
test_data.shape

In [ ]:
bids = torch.tensor([[
    [0., 1., 1.],
    [0., 1., 0.]
]]).to(device)
model.sinkhorn_rounds = 20
model.sinkhorn_epsilon = 3e-2
model(bids)

In [ ]:
cated = torch.cat((torch.ones((1, 3, 4)), torch.ones(1, 3, 4) * 2), dim=0)
cated[1, 0, 0] = 0 